In [2]:
from urlpath import URL
import requests
from pprint import pprint

import time
from IPython.display import clear_output

# url = URL("https://api.weather.gov/products/1eae2f39-8187-42ad-ab00-1f212b40a93a")
# url = URL("https://api.weather.gov/products/9c5a3929-9e59-46df-b406-a1aae6ca946c")
station = "MDW"
url = URL(f"https://forecast.weather.gov/product.php?site=DTX&issuedby={station}&product=CLI&format=CI&version=1&glossary=0")
while True:
    print(f"Query Time: {time.ctime()}")
    response = requests.get(url)
    response.raise_for_status()
    # print(response.text, file=open("../tmp/cli.html", 'w'))

    lines = response.text.splitlines()
    for i in range(len(lines) - 3):
        match_start = (f"CLI{station}" in lines[i] and "CLIMATE REPORT" in lines[i+2] and "NATIONAL WEATHER SERVICE" in lines[i+3])
        if match_start:
            match_start_i = i
            # print(i)
        match_end = "$$" in lines[i]
        if match_end:
            match_end_i = i
            # print(i)

    cli_text = "\n".join(lines[match_start_i:match_end_i+1])

    from src.nws import NWSClient

    report = NWSClient._parse_product_text(cli_text)
    report.raw_text = None
    print(f"Summary Date {report.summary_date.strftime('%Y-%m-%d')}")
    pprint(report)

    clear_output(wait=True)
    time.sleep(15)

KeyboardInterrupt: 